In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

import network_maps

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
plt.rcParams["figure.dpi"] = 200
fig_size = (32, 8)

In [ ]:
data_dir = 'data/2024-11-07-mikrotik/'

list_of_files = os.listdir(data_dir)
list_of_files.sort()
num_files = len(list_of_files)
print('Number of files: {}'.format(num_files))

df_list = []
for file in list_of_files:
    # Skipping the files we're not using
    if file[-5:] != ".gzip": # .gzip
        continue
    temp_df = pd.read_parquet(data_dir+file)
    df_list.append(temp_df)
df = pd.concat(df_list)

# Infer objects, then convert dtypes
df = df.infer_objects().convert_dtypes()

for column in df.columns:
        try:
            df[column] = pd.to_numeric(df[column])
        except (ValueError, TypeError):
            pass  # Skip columns that cannot be convertedmq

# Rename timestamp from Python and keep it for future use
# It is unreliable if a lot of messages come at the same time due to congestion
#df['timestamp_python'] = df.pop('@timestamp')

# It is better to rely on timestamps from the router rather than ElasticSearch
df['timestamp_router'] = pd.to_datetime(df['date'] + ' ' +  df['time'])
df.drop(columns=['date', 'time'], inplace=True)
df = df.sort_values(by=['timestamp_router'])
df = df.set_index('timestamp_router', drop=False)
#df = df.reset_index()

# Convert Data Class into integer mapping
dataclass_mapping = {'': 0, 'LTE': 1, '5G NSA': 2, '5G SA': 3}
df['lDataClassInt'] = df['lDataClass'].map(dataclass_mapping)

# Convert modulation into fixed bits per hz mapping
modulation_mapping = {'': 0, 'qpsk': 2, '16qam': 4, '64qam': 6, '256qam': 8}
df['lDlModulationInt'] = df['lDlModulation'].map(modulation_mapping)
df['lNrDlModulationInt'] = df['lNrDlModulation'].map(modulation_mapping)

# Fix wrong scaling on Rsrq and NrRsrq
# If the value is -12dB, it is shown as -120
df['lRsrq'] = df['lRsrq'] / 10
df['lNrRsrq'] = df['lNrRsrq'] / 10

# Create handover events
df['handover_CurrentCellid'] = df['lCurrentCellid'].diff().ne(0).astype(float)

print('df before')
df.info(verbose=True, show_counts=True, memory_usage='deep')

# Compress float64/Float64 to float32 and int64/Int64 to int32
dtype_mapping = {col: 'float32' for col in df.select_dtypes(include=['float64', 'Float64']).columns}
dtype_mapping.update({col: 'int32' for col in df.select_dtypes(include=['int64', 'Int64']).columns})

df = df.astype(dtype_mapping)

print('df after')
df.info(verbose=True, show_counts=True, memory_usage='deep')

#df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.keys()

In [ ]:
df['identity'].unique()

In [ ]:
df.columns = df.columns.str.removeprefix("gps.")
df.columns = df.columns.str.removeprefix("lte.")
df.info()

In [ ]:
df['lDataClass'].unique()

In [ ]:
df['lDataClass'].value_counts()

In [ ]:
df['lPrimaryBand'].unique()

In [ ]:
#df = df.between_time('11:41', '12:00')
# start_date = '2025-01-22 09:00:00'
# end_date = '2025-01-22 11:30:00'
# mask = (df['timestamp_router'] > start_date) & (df['timestamp_router'] <= end_date)
# df = df.loc[mask] 

In [ ]:
def get_PrimaryBandMHzNumber_v2(df):
    # Extract 'PrimaryBand' and 'PrimaryBandMHz' directly using regex
    df[['PrimaryBandName', 'PrimaryBandMHz']] = df['lPrimaryBand'].str.extract(r'(\S+)@(\d+Mhz)')
    
    # Drop rows where 'PrimaryBandMHz' is NaN
    #df.dropna(subset=['PrimaryBandMHz'], inplace=True)
    
    # Convert 'PrimaryBandMHz' to integer after removing 'Mhz'
    df['PrimaryBandMHz'] = df['PrimaryBandMHz'].str.replace('Mhz', '').astype(int)
    
    # Drop the intermediate columns
    df.drop(columns=['lPrimaryBand'], inplace=True)
    
    return df

In [ ]:
def get_PrimaryBandMHzNumber_v3(df):
    # Extract 'PrimaryBand' and 'PrimaryBandMHz' using regex
    df[['PrimaryBandName', 'PrimaryBandMHz']] = df['lPrimaryBand'].str.extract(r'(\S+)@(\d+)Mhz')
    
    # Convert 'PrimaryBandMHz' to integer, handling NaN values
    df['PrimaryBandMHz'] = df['PrimaryBandMHz'].astype('Int64')  # Nullable integer type
    
    # Drop the original column
    df.drop(columns=['lPrimaryBand'], inplace=True)
    
    return df


In [ ]:
df.info()

In [ ]:
df = get_PrimaryBandMHzNumber_v3(df)

In [ ]:
df['speed'] = pd.to_numeric(df['speed'].str.split().str[0])

In [ ]:
df['speed'].describe()

In [ ]:
df['lDataClass'].unique()

In [ ]:
df.head(10)

In [ ]:
df['identity'].unique()

In [ ]:
df_D2 = df.query("identity == '5G-D2-WAVELAB'")
df_DTAG = df.query("identity == '5G-DTAG-WAVELAB'")
df_8388 = df.query("identity == 'CAU-8388'")
df_4329 = df.query("identity == 'CAU-4329'")
df_0C = df.query("identity == 'CAU-0C'")

In [ ]:
df_D2['lDataClass'].unique()

In [ ]:
df_D2['PrimaryBandMHz'].plot(figsize=fig_size)

In [ ]:
df_DTAG['lDataClass'].unique()

In [ ]:
ax = df_D2[['lDataClassInt']].plot(figsize=fig_size, style='.-', title='D2')
ay = df_DTAG[['lDataClassInt']].plot(figsize=fig_size, style='.-', title='DTAG')

In [ ]:
ax = df_D2[['lCqi']].plot(figsize=fig_size, style='.-', title='D2')
ay = df_DTAG[['lCqi']].plot(figsize=fig_size, style='.-', title='DTAG')

In [ ]:
# SINR Range: -20 to 30 dB in LTE, -23 to 40 dB in 5G NR.

# <SINR>
# In LTE mode:
# It indicates LTE Signal-to-Interface plus Noise Ratio. 
# The conversion formula for actual SINR is 
# Y = (1/5) x X x 10 - 20 
# (X is the <SINR> value queried by AT+QENG and 
# Y is the actual value of LTE SINR after calculating with the formula). 
# Range: -20 to 30 dB.
# In 5G NR mode:
# It indicates the signal of 5G NR Signal-to-Interface plus Noise Ratio.
# Range: -20 to 30 dB.
df_D2[['lSinr']].describe()

In [ ]:
# Define the valid SINR range
sinr_min = -20  # Minimum SINR value
sinr_max = 30   # Maximum SINR value

df_D2['SINR_clipped'] = df_D2['lSinr'].clip(lower=-20, upper=30)

# Normalize SINR to range [0, 1]
df_D2['SINR_normalized'] = (df_D2['SINR_clipped'] - sinr_min) / (sinr_max - sinr_min)

In [ ]:
aa = df_D2[['lSinr']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
aa = df_D2[['SINR_normalized']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
# <RSRP> 16-bit signed integer.
# In LTE mode:
# It indicates the signal of LTE Reference Signal Received Power (see 3GPP 36.214). 
# Range: -140 to -44 dBm. The closer to -44, the better the signal is.
# The closer to -140, the worse the signal is.
# In 5G NR mode:
# It indicates the signal of 5G NR Reference Signal Received Power. 
# Range: -140 to -44 dBm. The closer to -44, the better the signal is. 
# The closer to -140, the worse the signal is.
df_D2[['lRsrp']].describe()

In [ ]:
# Define the valid RSRP range
rsrp_min = -140  # Minimum RSRP value
rsrp_max = -44   # Maximum RSRP value

df_D2['RSRP_clipped'] = df_D2['lRsrp'].clip(lower=rsrp_min, upper=rsrp_max)

# Normalize RSRP to range [0, 1]
df_D2['RSRP_normalized'] = (df_D2['RSRP_clipped'] - rsrp_min) / (rsrp_max - rsrp_min)

In [ ]:
aa = df_D2[['lRsrp']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
aa = df_D2[['RSRP_normalized']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
# <RSRQ> 
# In LTE mode:
# It indicates the signal of current LTE Reference Signal Received Quality (see 3GPP 36.214). 
# Range: -20 to -3 dB. The closer to -3, the better the signal is. 
# The closer to -20, the worse the signal is.
# In 5G NR mode:
# It indicates the signal of current 5G NR Reference Signal Received Quality.
# Range: -20 to -3 dB. 
# The closer to -3, the better the signal is. 
# The closer to -20, the worse the signal is. 
df_D2[['lRsrq']].describe()

In [ ]:
# Define the valid RSRQ range
rsrq_min = -20  # Minimum RSRQ value
rsrq_max = -3   # Maximum RSRQ value

df_D2['RSRQ_clipped'] = df_D2['lRsrq'].clip(lower=rsrq_min, upper=rsrq_max)

# Normalize RSRQ to range [0, 1]
df_D2['RSRQ_normalized'] = (df_D2['RSRQ_clipped'] - rsrq_min) / (rsrq_max - rsrq_min)

In [ ]:
aa = df_D2[['lRsrq']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
aa = df_D2[['RSRQ_normalized']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
df_D2[['ltxbitspersecond']].div(1024*1024).describe()

In [ ]:
df_DTAG[['ltxbitspersecond']].div(1024*1024).describe()

In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['speed'].rename('D2-GPS-speed').plot(ax=ax, )
df_DTAG['speed'].rename('DTAG-GPS-speed').plot(ax=ax, )

ax.set_title('GPS Speed over time')
ax.set_ylabel('GPS Speed (km/h)')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['ltxbitspersecond'].div(1024*1024).rename('D2-Tx').plot(ax=ax, )
df_DTAG['ltxbitspersecond'].div(1024*1024).rename('DTAG-Tx').plot(ax=ax, )

ax.set_title('Tx Rate over time')
ax.set_ylabel('Tx Rate [Mbps]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# repetition of above for clarity
aa = df_D2[['ltxbitspersecond']].div(1024*1024).plot(figsize=fig_size, style='.-', label='D2')
ab = df_D2[['SINR_normalized']].plot(figsize=fig_size, style='.-', label='D2')
ac = df_D2[['RSRP_normalized']].plot(figsize=fig_size, style='.-', label='D2')
ad = df_D2[['RSRQ_normalized']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
df_D2_selected = df_D2[['lDataClass',
            'lCqi',

            'lRsrp',
            'RSRP_normalized',
            
            'lRsrq',
            'RSRQ_normalized',

            'lSinr',
            'SINR_normalized',

            'ltxbitspersecond',
            'ltxpacketspersecond',
            ]].select_dtypes(include='number')


df_D2_selected.info()

In [ ]:
df_D2.drop(df_D2[df_D2['lat'] == 'none'].index, inplace = True)
df_D2.drop(df_D2[df_D2['lon'] == 'none'].index, inplace = True)

In [ ]:
df_D2['lat'] = pd.to_numeric(df_D2['lat'])
df_D2['lon'] = pd.to_numeric(df_D2['lon'])
#network_maps.plot_HeatMap(df_D2, 'D2', 'lCqi')

In [ ]:
df_D2['PrimaryBandName'].value_counts()

In [ ]:
df_D2['PrimaryBandMHz'].value_counts()

In [ ]:
df_D2['lDataClass'].value_counts()

In [ ]:
df_D2['lDataClass'].value_counts(normalize=True) * 100

In [ ]:
df_D2_LTE = df_D2.query("`lDataClass` == 'LTE'")
df_D2_5G_NSA = df_D2.query("`lDataClass` == '5G NSA'")
df_D2_5G_SA = df_D2.query("`lDataClass` == '5G SA'")

In [ ]:
df_D2_5G_SA.info()

In [ ]:
network_maps.create_coverage_squares_with_metric(df_D2, 'D2', 'auto', '2024-11-07', 'lCqi', lon_step = 0.0002, lat_step = 0.0001)
network_maps.create_coverage_squares_with_metric(df_D2_LTE, 'D2', 'LTE', '2024-11-07', 'lCqi', lon_step = 0.0002, lat_step = 0.0001)
network_maps.create_coverage_squares_with_metric(df_D2_5G_NSA, 'D2', '5G-NSA', '2024-11-07', 'lCqi', lon_step = 0.0002, lat_step = 0.0001)
network_maps.create_coverage_squares_with_metric(df_D2_5G_SA, 'D2', '5G-SA', '2024-11-07', 'lCqi', lon_step = 0.0002, lat_step = 0.0001)